In [3]:
import pandas as pd
import numpy as np
import pyreadstat  ##conda install conda-forge::pyreadstat
import pyodbc
import sqlalchemy as sa
from sqlalchemy.engine import URL


## Database driver
Please note that connecting to a database requires the correct driver
The ODBC Driver 17 for SQL server can be downloaded from here:
<br/>
<a href="https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver16">SQL Driver</a>

In [4]:
server = 'gisvt2024.c5umke60egby.us-east-1.rds.amazonaws.com' 
database = 'Workshop' 
username = 'admin' 
password = 'gisvaltechuser' 

connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=%s;DATABASE=%s;UID=%s;PWD=%s" % (server, database, username, password)

In [5]:
## Extract data from SQL database using pyodbc
# This database was removed from AWS after the conference ended.
# I am leaving the code as an example

cnxn = pyodbc.connect(connection_string)
cursor = cnxn.cursor()
query = "SELECT * FROM SampleData;"
df = pd.read_sql(query, cnxn)
cnxn.close()
df.sample(5)


C:\Users\Brad\AppData\Local\Temp\ipykernel_8220\3754743420.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)


,ParcelId,SaleDate,SalesPrice,Sqft,LandSize,Bathrooms,Quality,GarageSize,EffAge,NBHD
774,116983.0,2023-12-01,209835.0,1445.0,12886.0,2.5,BelowAverage,264.0,44.0,102.0
213,140035.0,2023-04-11,484156.0,1831.0,10089.0,3.5,AboveAverage,360.0,6.0,103.0
469,116870.0,2023-07-24,519645.0,2324.0,16749.0,2.0,AboveAverage,384.0,15.0,103.0
561,127574.0,2023-09-01,444049.0,2817.0,10497.0,3.5,Average,360.0,54.0,103.0
795,107955.0,2023-12-13,178724.0,1472.0,12230.0,2.0,BelowAverage,0.0,52.0,101.0


In [6]:
## Extract data from SQL database using sqlalchemy

connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

from sqlalchemy import create_engine
engine = create_engine(connection_url)

with engine.begin() as conn:
    data = pd.read_sql_query(sa.text("SELECT * FROM SampleData;"), conn)

data.sample(5)

,ParcelId,SaleDate,SalesPrice,Sqft,LandSize,Bathrooms,Quality,GarageSize,EffAge,NBHD
705,127975.0,2023-11-06,345797.0,2099.0,15817.0,2.5,BelowAverage,384.0,0.0,103.0
625,108710.0,2023-09-28,174971.0,1189.0,17774.0,2.0,Average,264.0,58.0,101.0
359,102374.0,2023-06-16,472429.0,2245.0,12035.0,3.0,Average,384.0,1.0,103.0
529,122666.0,2023-08-19,284976.0,1689.0,23373.0,3.5,BelowAverage,360.0,34.0,102.0
497,121287.0,2023-08-03,678954.0,2781.0,33739.0,3.0,AboveAverage,704.0,38.0,104.0


In [ ]:
## Read SPSS sav file using read_spss

data = pd.read_spss('..\\SPSS\\Data\\GISValTechSampleData.sav')
data.head()
data['ParcelId'] = data['ParcelId'].astype(int)
data['SaleDate'] = pd.to_datetime(data['SaleDate'])
data['SalesPrice'] = data['SalesPrice'].astype(int)
data['Sqft'] = data['Sqft'].astype(int)
data['LandSize'] = data['LandSize'].astype(int)
data['Quality'] = pd.Categorical(data['Quality'], ordered = True, categories = [
    'Poor', 'BelowAverage', 'Average', 'AboveAverage', 'Superior'])
data['GarageSize'] = data['GarageSize'].astype(int)
data['EffAge'] = data['EffAge'].astype(int)
data['NBHD'] = pd.Categorical(data['NBHD'].astype(int))

data.set_index('ParcelId', inplace = True)
data.head()

In [7]:
## Read pickle file

data = pd.read_pickle('..\\datafiles\\SampleData.pkl')
data.sample(5)

,SaleDate,SalesPrice,Sqft,LandSize,Bathrooms,Quality,GarageSize,EffAge,NBHD
ParcelId,,,,,,,,,
114513,2023-06-01,117953,1126,14199,2.0,Poor,0,59,101
137252,2023-02-07,207578,1111,11554,3.0,BelowAverage,384,22,102
113664,2023-07-18,320760,1579,24023,2.0,BelowAverage,484,12,102
132662,2023-06-19,999232,4874,14171,4.5,AboveAverage,360,18,105
123900,2023-11-15,348404,1606,23086,2.5,Average,384,27,102


In [8]:
## Read excel file

data = pd.read_excel('..\\datafiles\\SampleData.xlsx')
data.sample(5)

,ParcelId,SaleDate,SalesPrice,Sqft,LandSize,Bathrooms,Quality,GarageSize,EffAge,NBHD
405,134390,2023-06-28,1050650,3979,17321,4.5,AboveAverage,704,11,105
707,124110,2023-11-07,463583,1968,23985,3.0,Average,360,55,104
547,103662,2023-08-26,314445,1625,15084,1.5,Average,0,33,101
630,104853,2023-10-01,339796,1873,10829,3.0,Average,384,24,103
382,135761,2023-06-23,419920,2241,19829,2.5,Average,0,1,103


In [9]:
## Read csv file
data = pd.read_csv('..\\datafiles\\SampleData.csv')
data.sample(5)

,ParcelId,SaleDate,SalesPrice,Sqft,LandSize,Bathrooms,Quality,GarageSize,EffAge,NBHD
625,108710,2023-09-28,174971,1189,17774,2.0,Average,264,58,101
400,114321,2023-06-28,486777,2308,15901,2.5,Average,264,10,103
165,137026,2023-03-09,1367323,4638,15639,4.0,Superior,484,15,105
334,131725,2023-06-02,170506,1213,12539,2.5,BelowAverage,264,52,102
78,132543,2023-01-20,758753,2347,29398,3.5,Superior,704,37,104


In [10]:
## Export data to SQL using pyodbc
cnxn = pyodbc.connect(connection_string)
cursor = cnxn.cursor()
cursor.execute('''
    IF OBJECT_ID (N'dbo.pyodbcTestData', N'U') IS NULL  
        CREATE TABLE dbo.pyodbcTestData(
            ParcelId INT,
            SaleDate DATETIME,
            SalesPrice INT
        );
''')
for index, row in data.loc[:5].iterrows():
    cursor.execute("INSERT INTO dbo.pyodbcTestData (ParcelId, SaleDate, SalesPrice) values (?, ?, ?)", row['ParcelId'], row['SaleDate'], row['SalesPrice'])
cnxn.commit()
cursor.close()
cnxn.close()


In [ ]:
## Export data to SQL using sqlalchemy
data[['ParcelId', 'SaleDate', 'SalesPrice']].to_sql(name='alchemyTestData', if_exists="replace", con=engine)

In [ ]:
## Export data to Excel
data.to_excel('..\\datafiles\\ExcelExport.xlsx')

In [ ]:
## Export data to csv
data.to_csv('..\\datafiles\\CsvExport.csv')

In [ ]:
## Export data to a pickle file
data.to_pickle('..\\datafiles\\PickleExport.pkl')